## Reestructuración de datos

Este notebook estructura y unifica el csv generado del scraper


In [28]:
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
uber = pd.read_csv('results/uber_results_geo.csv')
uber = uber.query('latitude.notna()').reset_index(drop=True)
rappi = pd.read_csv('results/rappi_results_geo.csv')
rappi = rappi.query('latitude.notna()').reset_index(drop=True)
rappi_processed = rappi.copy()
rappi_processed['schedule'] = rappi_processed['schedule'].str.replace(r'[\[\]]', '', regex=True)
rappi_processed['schedule'] = rappi_processed['schedule'].str.replace(r"'", '', regex=True).str.replace('Horarios de Apertura y Cierre', '')


# Funcion de estandaarización de horario rappi

In [29]:
def parse_schedule(schedule_str):
    """
    Parse restaurant opening hours from a schedule string.
    
    Args:
        schedule_str (str): String with schedule information in format like 
                           'Martes10:00 - 20:00Jueves10:00 - 20:00...'
    
    Returns:
        dict or None: Dictionary with days as keys and schedules as values.
                     Returns None if no schedule is found.
    """
    if not isinstance(schedule_str, str) or not schedule_str.strip():
        return None
    
    # Dictionary to store the schedules
    days_schedule = {
        'Lunes': None,
        'Martes': None,
        'Miércoles': None,
        'Jueves': None,
        'Viernes': None,
        'Sábado': None,
        'Domingo': None
    }
    
    # List of days to check
    days = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    
    # Check if any day is present in the schedule string
    found_any = False
    
    for i, day in enumerate(days):
        if day in schedule_str:
            found_any = True
            # Find the position of the current day
            start_pos = schedule_str.find(day) + len(day)
            
            # Find the position of the next day (if any)
            next_day_pos = float('inf')
            for next_day in days:
                pos = schedule_str.find(next_day, start_pos)
                if pos != -1 and pos < next_day_pos:
                    next_day_pos = pos
            
            # Extract the schedule for the current day
            if next_day_pos != float('inf'):
                schedule = schedule_str[start_pos:next_day_pos].strip()
            else:
                schedule = schedule_str[start_pos:].strip()
            
            days_schedule[day] = schedule
    
    # If no schedule was found for any day, return None
    if not found_any:
        return None
    
    return days_schedule

In [30]:
horarios_rappi = pd.DataFrame(rappi_processed['schedule'].apply(parse_schedule).tolist()).fillna('cerrado')
rappi_processed = pd.concat([rappi_processed, horarios_rappi], axis=1)
rappi_processed.rename(columns={'href': 'url','restaurantName':'nombre',"address":"direccion",}, inplace=True)
rappi_processed.rename(columns = {'Lunes':'lunes','Martes':'martes','Miércoles':'miercoles','Jueves':'jueves','Viernes':'viernes','Sábado':'sabado','Domingo':'domingo'}, inplace=True)
rappi_processed['geometry']= rappi_processed.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
rappi_processed = gpd.GeoDataFrame(rappi_processed, geometry='geometry',crs='EPSG:4326')
rappi_processed.drop(columns=['schedule','latitude','longitude'], inplace=True)

## Procesamieto Uber

In [31]:
import re
import json
uber_processed = uber.copy()
uber_processed['tags'] = uber_processed['tags'].str.replace(' • Información', '')
# Separa informacion y obtiene rating, rating_count, price_range y tag_list
def process_tags(df):
    """
    Process the 'tags' column of a DataFrame to extract:
    1. Rating: Number before "xStar"
    2. Rating count: Number in parentheses after "xStar"
    3. Price range: $ symbols
    4. Tags: List of words separated by •
    
    Args:
        df (pandas.DataFrame): DataFrame containing a 'tags' column
        
    Returns:
        pandas.DataFrame: DataFrame with new columns added
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Extract rating (number before "xStar")
    result_df['rating'] = result_df['tags'].str.extract(r'([\d\.]+)\s*xStar', expand=False).astype(float)
    
    # Extract rating count (number in parentheses after "xStar")
    result_df['rating_count'] = result_df['tags'].str.extract(r'xStar\s*\(([\d,\+]+)\)', expand=False)
    
    # Extract price range ($ symbols)
    def extract_price(tag_string):
        if pd.isna(tag_string):
            return None
        # Find all $ sequences
        price_matches = re.findall(r'([$]+)', tag_string)
        # Return the last one (which is typically at the end of the string)
        return price_matches[-1] if price_matches else None
    
    result_df['price_range'] = result_df['tags'].apply(extract_price)
    
    # Extract tags (words separated by •)
    def extract_tags(tag_string):
        if pd.isna(tag_string):
            return []
        # Split by • and remove any parts that contain "xStar", price symbols, or are empty
        parts = [part.strip() for part in tag_string.split('•')]
        # Filter out parts containing rating info or just price symbols
        filtered_parts = [part for part in parts 
                         if part and 'xStar' not in part and not re.match(r'^[$]+$', part)]
        return filtered_parts
    
    result_df['tag_list'] = result_df['tags'].apply(extract_tags)
    
    return result_df

# Apply the function to the uber DataFrame
uber_processed = process_tags(uber_processed)
uber_processed = uber_processed.drop(columns=['tags'])
uber_processed['address'] = uber_processed['address'].str.replace('\n\n', ' ')

# Crea diccionario a partir de un rango de dias
def creaDict(fDay, lDay, schedule):
    dias = [
        "domingo",
        "lunes",
        "martes",
        "miércoles",
        "jueves",
        "viernes",
        "sábado",
    ]

    if fDay not in dias or lDay not in dias:
        raise ValueError("fDay y lDay deben ser días válidos de la semana")

    i = dias.index(fDay)
    j = dias.index(lDay)

    # Si no hay wrap-around
    if i <= j:
        dias_rango = dias[i : j + 1]
    # Si hay wrap-around (ej. viernes → lunes)
    else:
        dias_rango = dias[i:] + dias[: j + 1]

    return {dia: schedule for dia in dias_rango}

# Crea diccionario final
def creaDictFinal(dicts):
    dias = [
        "domingo",
        "lunes",
        "martes",
        "miércoles",
        "jueves",
        "viernes",
        "sábado",
    ]

    resultado = {}

    for dia in dias:
        valores = []

        for d in dicts:
            if dia in d:
                valores.append(d[dia])

        if valores:
            valores = list(dict.fromkeys(valores))
            resultado[dia] = " | ".join(valores)
        else:
            resultado[dia] = "cerrado"

    return resultado

DAY = r"(?:domingo|lunes|martes|mi[eé]rcoles|jueves|viernes|s[áa]bado)"
# REGEX PARA EXTRAER CONJUNTO DE DIAS
RANGE_PAT = re.compile(
    rf"{DAY}\s*-\s*{DAY}"                             # inicio: Día - Día
    rf"(?:(?!"                                       # consumir mientras NO aparezca:
    rf"{DAY}\s*-\s*{DAY}"                            # 1) otro Día - Día
    rf"|{DAY}\s*\d{{1,2}}:"                           # 2) Día + 1-2 dígitos + :
    rf"|{DAY}"                                        # 3) Día 
    rf").)*",
    flags=re.DOTALL
)
# REGEX PARA EXTRAER DIAS
DAY_PAT = re.compile(rf"{DAY}",flags=re.DOTALL)
# REGEX PARA EXTRAER DIAS
DAY_TIME_PAT = re.compile(
    rf"(?!\s)"                                        #  no puede iniciar con espacio
    rf"{DAY}\s*\d{{1,2}}:"                             # inicio: Día + hora
    rf"(?:(?!"                                        # consumir mientras NO aparezca:
    rf"{DAY}\s*-\s*{DAY}"                             # 1) otro Día - Día
    rf"|{DAY}\s*\d{{1,2}}:"                            # 2) Día + 1-2 dígitos + :
    rf"|{DAY}"                                        # 3) Día
    rf").)*",
    flags=re.DOTALL
)
# REGEX PARA EXTRAER HORARIOS
TIME_PAT = re.compile(
    r"(?:\d{1,2}:\d{1,2}\s*(?:a\.m\.|p\.m\.)\s*-\s*\d{1,2}:\d{1,2}\s*(?:a\.m\.|p\.m\.))"
    r"(?:(?!(?:\d{1,2}:\d{1,2}\s*(?:a\.m\.|p\.m\.)\s*-\s*\d{1,2}:\d{1,2}\s*(?:a\.m\.|p\.m\.))).)*",
    re.DOTALL
)


def sortSchedule(schedule):
    if not isinstance(schedule, str):
        return creaDict('domingo','sábado',None)
    final = []
    schedule = schedule.lower()
    # BUSQUEDA DE TODOS LOS DÍAS
    if "todos los días" in schedule:
        horarios = [m.group(0).replace('\xa0•\xa0',' ') for m in TIME_PAT.finditer(schedule)]
        horarios = " | ".join(horarios)
        idict = creaDict("domingo","sábado",horarios)
        final.append(idict)
        final = creaDictFinal(final)
        return final
    # BUSQUEDA DE RANGOS DE DÍAS
    ranges = [m.group(0) for m in RANGE_PAT.finditer(schedule.lower())]
    for match in ranges:
        days = [m.group(0) for m in DAY_PAT.finditer(match.lower())]     
        horarios = [m.group(0).replace('\xa0•\xa0',' ') for m in TIME_PAT.finditer(match.lower())]
        horarios = " | ".join(horarios)
        idict = creaDict(days[0],days[1],horarios)
        final.append(idict)
    # BUSQUEDA DE DIAS ÚNICOS
    new_text = " ".join(re.split(RANGE_PAT,schedule))
    days = [m.group(0) for m in DAY_TIME_PAT.finditer(new_text.lower())]    
    for day in days:  
        horarios = [m.group(0).replace('\xa0•\xa0',' ') for m in TIME_PAT.finditer(day.lower())]
        dayn = [m.group(0) for m in DAY_PAT.finditer(day.lower())]     
        horarios = " | ".join(horarios)
        final.append({dayn[0]:horarios})
    final = creaDictFinal(final)
    return final


In [32]:

schedule_data = uber_processed['schedule'].apply(sortSchedule).tolist()
# Convertir la lista de diccionarios a un DataFrame
schedule_df_uber = pd.DataFrame(schedule_data)
# # Añadir las columnas de horario al DataFrame uber_processed
uber_processed = pd.concat([uber_processed, schedule_df_uber], axis=1)
uber_processed['geometry']= uber_processed.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
uber_processed = gpd.GeoDataFrame(uber_processed, geometry='geometry',crs='EPSG:4326')
uber_processed.rename(columns={'href': 'url','restaurantName':'nombre','address':'direccion','rating_count':'contRating','price_range':'rangPrec',"tag_list":'tags' }, inplace=True)
uber_processed.rename(columns = {'miércoles':'miercoles','sábado':'sabado'}, inplace=True)
uber_processed.drop(columns=['schedule','latitude','longitude'], inplace=True)


## Concatenación de resultados

In [33]:
final = gpd.GeoDataFrame(pd.concat([rappi_processed,uber_processed]),geometry='geometry')
final.to_file("results/completos/comercios_delivery.gpkg",driver='GPKG')
final.to_csv("results/completos/comercios_delivery.csv",index=False)


In [35]:
final

,url,nombre,tags,direccion,rating,lunes,martes,miercoles,jueves,viernes,sabado,domingo,geometry,contRating,rangPrec
0,https://www.rappi.com.mx/restaurantes/19237591...,Caldos de Gallina Esnarlin - Unidad Hab Santa ...,Pollo,Calle 71 esquina avenida 12 samuel gomper Sant...,3.7,08:15 - 18:45,08:15 - 18:45,00:00 - 00:00,cerrado,08:15 - 18:45,08:15 - 18:45,08:15 - 18:45,POINT (-99.04752 19.35059),NaN,NaN
1,https://www.rappi.com.mx/restaurantes/19237805...,Ham & Jam Cafetería - Unidad Hab San Juan Xalpa,Postres,"Av San Lorenzo 2171, San Juan Xalpa, Iztapalap...",NaN,08:30 - 20:30,08:30 - 20:30,cerrado,cerrado,08:30 - 20:30,08:30 - 20:30,cerrado,POINT (-99.07518 19.3345),NaN,NaN
2,https://www.rappi.com.mx/restaurantes/19237975...,Avocalia - Los Ángeles,Saludables y Ensaladas,"Av. del Rosal 196, Los Ángeles, Iztapalapa, 09...",NaN,07:00 - 22:30,07:00 - 22:30,07:30 - 15:30,10:00 - 22:00,07:00 - 23:30,07:00 - 23:30,07:00 - 22:30,POINT (-99.06742 19.34475),NaN,NaN
3,https://www.rappi.com.mx/restaurantes/19238014...,Tupastelerialosglobos - Santa María Aztahuacán,Panadería y Pastelería,"Esquina Calle 20 de Noviembre, Anillo de Circu...",4.2,08:30 - 20:40,08:30 - 20:40,cerrado,cerrado,08:30 - 20:40,08:30 - 20:40,08:30 - 20:40,POINT (-99.0324 19.34876),NaN,NaN
4,https://www.rappi.com.mx/restaurantes/19238019...,Brunch & Munch - Los Ángeles,Desayunos y Brunch,"Av. del Rosal 196, Los Ángeles, Iztapalapa, 09...",NaN,07:00 - 22:30,07:00 - 22:30,07:30 - 15:30,10:00 - 22:00,07:00 - 23:30,07:00 - 23:30,07:00 - 22:30,POINT (-99.06742 19.34475),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,https://www.ubereats.com/mx/store/comida-japon...,Comida Japonesa JOSHI,"[Asiática, Asiática fusión, Especialidades]","Av Valle del Yukon 112, Valle de Aragon 1ra Se...",4.5,2:00 p.m. - 8:00 p.m.,,,2:00 p.m. - 8:00 p.m.,2:00 p.m. - 8:00 p.m.,2:00 p.m. - 8:00 p.m.,2:00 p.m. - 8:00 p.m.,POINT (-99.05896 19.49023),"2,000+",$
1993,https://www.ubereats.com/mx/store/zamaburgers-...,ZamaBurgers (Mexico),"[Hamburguesas, Americana, Sándwiches, Comida e...","Calle Tezoquipa 15 Tlalpan, DF",4.6,,,6:30 p.m. - 10:30 p.m.,7:00 p.m. - 10:30 p.m.,7:15 p.m. - 10:30 p.m.,10:15 a.m. - 11:00 p.m.,10:00 a.m. - 9:30 p.m.,POINT (-99.16795 19.2832),4,None
1994,https://www.ubereats.com/mx/store/churros-juan...,Churros Juanes,"[Mexicana, Latinoamericana, Mexicana]","13 Avenida Luis Hidalgo Monroy 380 Iztapalapa,...",4.5,4:00 p.m. - 11:00 p.m.,4:00 p.m. - 11:00 p.m.,4:00 p.m. - 11:00 p.m.,4:00 p.m. - 11:00 p.m.,4:00 p.m. - 11:00 p.m.,4:00 p.m. - 11:00 p.m.,4:00 p.m. - 11:00 p.m.,POINT (-99.07528 19.35443),110+,$
1995,https://www.ubereats.com/mx/store/torteria-col...,Tortería Colima,"[Mexicana, Latinoamericana, Mexicana]","Tecamachalco 79 Naucalpan de Juárez, LATAM",4.7,9:00 a.m. - 9:00 p.m.,9:00 a.m. - 9:00 p.m.,9:00 a.m. - 9:00 p.m.,9:00 a.m. - 9:00 p.m.,9:00 a.m. - 9:00 p.m.,9:00 a.m. - 9:00 p.m.,9:00 a.m. - 9:00 p.m.,POINT (-99.23054 19.42484),410+,$
